In [1]:
import os
import polars as pl

In [2]:
current_dir = os.getcwd()
scripts = os.path.dirname(current_dir)
parent_dir = os.path.dirname(scripts)
output_path = os.path.join(parent_dir, "data", "tennis_dataset_app.csv")

In [3]:
dataset_path = os.path.join(parent_dir, "data", "tennis_dataset_clean_essaie.csv")
df = pl.read_csv(dataset_path)
df = df.with_columns(
    pl.col("date").str.strptime(pl.Date, format="%d.%m.%y").alias("date")
)
df

index,player1_name,player1_age,player1_ranking,player1_points,player1_win_rate,player1_win_rate_3_sets,player1_win_rate_tiebreak,player1_total_matches_dure,player1_win_rate_dure,player1_total_matches_terre battue,player1_win_rate_terre battue,player1_total_matches_gazon,player1_win_rate_gazon,player1_total_matches_salle,player1_win_rate_salle,player1_total_matches_carpet,player1_win_rate_carpet,player1_total_matches_acryl,player1_win_rate_acryl,player1_avg_first_serve_pct,player1_avg_first_serve_won_pct,player1_avg_second_serve_won_pct,player1_avg_return_points_won_pct,player1_avg_break_point_won_pct,player1_avg_double_fautes,player1_avg_aces,player2_name,player2_age,player2_ranking,player2_points,player2_win_rate,player2_win_rate_3_sets,player2_win_rate_tiebreak,player2_total_matches_dure,player2_win_rate_dure,player2_total_matches_terre battue,player2_win_rate_terre battue,player2_total_matches_gazon,player2_win_rate_gazon,player2_total_matches_salle,player2_win_rate_salle,player2_total_matches_carpet,player2_win_rate_carpet,player2_total_matches_acryl,player2_win_rate_acryl,player2_avg_first_serve_pct,player2_avg_first_serve_won_pct,player2_avg_second_serve_won_pct,player2_avg_return_points_won_pct,player2_avg_break_point_won_pct,player2_avg_double_fautes,player2_avg_aces,tournament_category,target,date,surface_dure,surface_salle,surface_terre_batue,surface_gazon,ranking_diff,points_diff,win_rate_diff,win_rate_diff_3_sets,win_rate_tiebreak_diff,aces_diff,double_faults_diff,win_rate_dure_diff,win_rate_terre_diff,win_rate_gazon_diff,win_rate_salle_diff
i64,str,i64,i64,i64,f64,f64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,str,i64,i64,i64,f64,f64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,i64,date,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,"""Lucas Pouille""",30,101,616,0.675676,0.6,0.769231,12,0.75,13,0.615385,6,0.833333,6,0.5,0,0,0,0,0.569359,0.740458,0.53238,0.38835,0.405622,3.166667,6.833333,"""Gijs Brouwer""",28,176,326,0.565217,0.666667,0.375,18,0.555556,11,0.545455,12,0.583333,5,0.6,0,0,0,0,0.684263,0.699219,0.512169,0.385107,0.382979,1.5,3.826087,2,0,2024-09-22,1,0,0,0,75,290,0.110458,0.066667,0.0,3.007246,1.666667,0.194444,0.06993,0.25,0.1
1,"""Taylor Harry Fritz""",27,4,5100,0.75,0.727273,0.75,0,0.0,8,0.625,12,0.833333,0,0.0,0,0,0,0,0.634857,0.799842,0.559835,0.348439,0.4,1.590909,12.863636,"""Jack Draper""",23,15,2685,0.56,0.466667,0.555556,4,0.25,11,0.454545,10,0.8,0,0.0,0,0,0,0,0.604895,0.792733,0.493047,0.360829,0.395833,3.24,9.08,3,1,2024-07-30,0,0,1,0,11,2415,0.19,0.260606,0.0,3.783636,1.649091,0.25,0.170455,0.033333,0.0
2,"""Tommy Paul""",27,12,3145,0.711111,0.576923,0.916667,18,0.611111,14,0.714286,12,0.833333,1,1.0,0,0,0,0,0.616502,0.713651,0.555318,0.412325,0.397849,1.733333,5.177778,"""Miomir Kecmanovic""",25,54,1021,0.45,0.444444,0.461538,17,0.294118,12,0.5,8,0.5,3,1.0,0,0,0,0,0.622523,0.710345,0.529309,0.359486,0.391667,2.102564,4.230769,1,1,2024-10-18,0,1,0,0,42,2124,0.261111,0.132479,0.0,0.947009,0.369231,0.316993,0.214286,0.333333,0.0
3,"""Holger Rune""",21,13,3025,0.625,0.625,0.666667,18,0.611111,17,0.647059,5,0.6,0,0.0,0,0,0,0,0.596603,0.724453,0.542105,0.349387,0.396552,3.075,5.775,"""Nicolas Jarry""",29,35,1370,0.454545,0.5,0.6,19,0.421053,18,0.5,1,0.0,6,0.5,0,0,0,0,0.632006,0.746787,0.507726,0.329688,0.334802,2.272727,9.022727,2,1,2024-10-22,0,1,0,0,22,1655,0.170455,0.125,0.0,3.247727,0.802273,0.190058,0.147059,0.6,0.5
5,"""Damir Dzumhur""",32,83,679,0.678571,0.642857,0.5,5,0.6,18,0.777778,5,0.4,0,0.0,0,0,0,0,0.685901,0.639356,0.523962,0.432635,0.445344,2.285714,0.535714,"""Jesper De Jong""",24,112,548,0.71875,0.571429,0.857143,6,0.833333,24,0.708333,2,0.5,0,0.0,0,0,0,0,0.564008,0.730239,0.513133,0.416467,0.442379,3.3125,5.53125,2,1,2024-09-07,1,0,0,0,29,131,0.040179,0.071429,0.0,4.995536,1.026786,0.233333,0.069444,0.1,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…

In [4]:
joueurs = set(list(df.select(pl.col("player1_name")).to_numpy().flatten()) + list(df.select(pl.col("player2_name")).to_numpy().flatten()))
assert len(joueurs) == 200, f"nombres de joueurs incomplet, joueurs = {len(joueurs)}"

In [5]:
derniers_matches = pl.DataFrame()
for joueur in joueurs:
    dernier_match = (
        df.filter(
            (pl.col("player1_name") == joueur) | (pl.col("player2_name") == joueur)
        )
        .sort(by="date", descending=True)
        .head(1)
    )
    if derniers_matches.is_empty():
        derniers_matches = dernier_match
    else:
        derniers_matches = pl.concat([derniers_matches, dernier_match], how="vertical")

In [6]:
derniers_matches.write_csv(output_path)

In [8]:
derniers_matches.unique("player1_name")

index,player1_name,player1_age,player1_ranking,player1_points,player1_win_rate,player1_win_rate_3_sets,player1_win_rate_tiebreak,player1_total_matches_dure,player1_win_rate_dure,player1_total_matches_terre battue,player1_win_rate_terre battue,player1_total_matches_gazon,player1_win_rate_gazon,player1_total_matches_salle,player1_win_rate_salle,player1_total_matches_carpet,player1_win_rate_carpet,player1_total_matches_acryl,player1_win_rate_acryl,player1_avg_first_serve_pct,player1_avg_first_serve_won_pct,player1_avg_second_serve_won_pct,player1_avg_return_points_won_pct,player1_avg_break_point_won_pct,player1_avg_double_fautes,player1_avg_aces,player2_name,player2_age,player2_ranking,player2_points,player2_win_rate,player2_win_rate_3_sets,player2_win_rate_tiebreak,player2_total_matches_dure,player2_win_rate_dure,player2_total_matches_terre battue,player2_win_rate_terre battue,player2_total_matches_gazon,player2_win_rate_gazon,player2_total_matches_salle,player2_win_rate_salle,player2_total_matches_carpet,player2_win_rate_carpet,player2_total_matches_acryl,player2_win_rate_acryl,player2_avg_first_serve_pct,player2_avg_first_serve_won_pct,player2_avg_second_serve_won_pct,player2_avg_return_points_won_pct,player2_avg_break_point_won_pct,player2_avg_double_fautes,player2_avg_aces,tournament_category,target,date,surface_dure,surface_salle,surface_terre_batue,surface_gazon,ranking_diff,points_diff,win_rate_diff,win_rate_diff_3_sets,win_rate_tiebreak_diff,aces_diff,double_faults_diff,win_rate_dure_diff,win_rate_terre_diff,win_rate_gazon_diff,win_rate_salle_diff
i64,str,i64,i64,i64,f64,f64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,str,i64,i64,i64,f64,f64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,i64,date,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
449,"""Jaime Faria""",21,123,489,0.591837,0.571429,0.642857,7,0.571429,38,0.657895,2,0.0,2,0.0,0,0,0,0,0.563352,0.709213,0.486056,0.388621,0.443425,4.595745,5.659574,"""Roman Andres Burruchaga""",22,156,378,0.604167,0.590909,1.0,4,0.5,41,0.609756,3,0.666667,0,0.0,0,0,0,0,0.602881,0.660653,0.475944,0.433544,0.480337,3.5,2.020833,2,0,2024-12-31,1,0,0,0,33,111,0.01233,0.019481,0.0,3.638741,1.095745,0.071429,0.048139,0.666667,0.0
1959,"""Marin Cilic""",36,179,323,0.55102,0.583333,0.631579,26,0.615385,5,0.4,4,0.75,14,0.428571,0,0,0,0,0.583619,0.770089,0.498755,0.360981,0.375405,4.23913,11.869565,"""Jiri Lehecka""",23,28,1660,0.636364,0.647059,0.833333,26,0.692308,5,0.8,0,0.0,13,0.461538,0,0,0,0,0.617106,0.757079,0.522018,0.351897,0.368627,2.704545,8.25,1,0,2024-11-05,0,1,0,0,151,1337,0.085343,0.063725,0.0,3.619565,1.534585,0.076923,0.4,0.75,0.032967
1675,"""Lukas Klein""",26,136,443,0.530612,0.363636,0.75,18,0.5,16,0.5625,5,0.6,10,0.5,0,0,0,0,0.652208,0.741281,0.519392,0.363682,0.353312,3.0625,6.142857,"""Li Tu""",28,174,329,0.604167,0.578947,0.7,35,0.657143,2,0.5,9,0.444444,2,0.5,0,0,0,0,0.628272,0.714216,0.531069,0.395906,0.413559,2.25,5.5,1,0,2024-12-28,1,0,0,0,38,114,0.073554,0.215311,0.0,0.642857,0.8125,0.157143,0.0625,0.155556,0.0
1378,"""Alejandro Tabilo""",27,23,1943,0.530612,0.47619,0.55,12,0.416667,18,0.611111,9,0.777778,10,0.3,0,0,0,0,0.639082,0.748286,0.489286,0.348433,0.379592,2.510204,7.367347,"""Benjamin Bonzi""",28,75,730,0.770833,0.615385,0.823529,19,0.631579,0,0.0,2,0.5,27,0.888889,0,0,0,0,0.609009,0.763807,0.548387,0.396237,0.415033,2.125,6.75,1,0,2024-12-30,1,0,0,0,52,1213,0.240221,0.139194,0.0,0.617347,0.385204,0.214912,0.611111,0.277778,0.588889
2189,"""Roman Safiullin""",27,60,923,0.55102,0.380952,0.571429,23,0.608696,13,0.384615,5,0.4,8,0.75,0,0,0,0,0.610916,0.736475,0.503218,0.373294,0.380952,3.081633,6.918367,"""Fabian Marozsan""",25,58,935,0.425532,0.473684,0.733333,21,0.571429,13,0.384615,5,0.2,8,0.25,0,0,0,0,0.645512,0.687777,0.483065,0.363881,0.404,2.195652,4.326087,1,0,2024-12-30,1,0,0,0,2,12,0.125488,0.092732,0.0,2.59228,0.88598,0.037267,0.0,0.2,0.5
…,…,…,…,…,…,…,…,…,…,…,

In [ ]:
df_player1 = derniers_matches.select([col for col in derniers_matches.columns if "player1" in col] + ["date"])

In [29]:
df_player1 = derniers_matches.select([col for col in derniers_matches.columns if "player1" in col] + ["date"])
df_player2 = derniers_matches.select([col for col in derniers_matches.columns if "player2" in col] + ["date"]).rename({col: col.replace("player2", "player1") for col in derniers_matches.columns})

In [36]:
df_combined = pl.concat([df_player1, df_player2], how="vertical")
df_combined = df_combined.sort(by="date", descending=True)
df_recent = df_combined.group_by("player1_name").agg(pl.col("*").first())
df_recent

player1_name,player1_age,player1_ranking,player1_points,player1_win_rate,player1_win_rate_3_sets,player1_win_rate_tiebreak,player1_total_matches_dure,player1_win_rate_dure,player1_total_matches_terre battue,player1_win_rate_terre battue,player1_total_matches_gazon,player1_win_rate_gazon,player1_total_matches_salle,player1_win_rate_salle,player1_total_matches_carpet,player1_win_rate_carpet,player1_total_matches_acryl,player1_win_rate_acryl,player1_avg_first_serve_pct,player1_avg_first_serve_won_pct,player1_avg_second_serve_won_pct,player1_avg_return_points_won_pct,player1_avg_break_point_won_pct,player1_avg_double_fautes,player1_avg_aces,date
str,i64,i64,i64,f64,f64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,date
"""Fabio Fognini""",37,91,637,0.522727,0.590909,0.5,8,0.375,26,0.576923,5,0.6,5,0.4,0,0,0,0,0.581429,0.697512,0.472702,0.380311,0.448052,4.659091,3.522727,2024-11-05
"""Federico Coria""",32,100,617,0.55102,0.416667,0.615385,2,0.0,46,0.586957,1,0.0,0,0.0,0,0,0,0,0.669214,0.614192,0.481023,0.444413,0.463768,2.458333,1.895833,2024-11-30
"""Yoshihito Nishioka""",29,69,776,0.55102,0.421053,0.714286,30,0.6,12,0.333333,7,0.714286,0,0.0,0,0,0,0,0.634351,0.673585,0.561375,0.364076,0.375427,1.222222,2.711111,2025-01-01
"""Emil Ruusuvuori""",25,95,628,0.510204,0.52381,0.75,17,0.588235,8,0.25,6,0.5,18,0.555556,0,0,0,0,0.633367,0.700437,0.551273,0.366565,0.363333,2.25,4.833333,2024-07-31
"""Dominik Koepfer""",30,102,610,0.55102,0.6,0.4375,16,0.625,15,0.4,4,0.5,14,0.642857,0,0,0,0,0.622751,0.701778,0.529758,0.366515,0.366242,3.083333,4.354167,2024-12-30
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Nicolas Jarry""",29,35,1370,0.469388,0.5,0.608696,21,0.47619,18,0.5,1,0.0,9,0.444444,0,0,0,0,0.631618,0.75089,0.511186,0.327901,0.342857,2.204082,8.938776,2025-01-03
"""Yasutaka Uchiyama""",32,146,398,0.583333,0.681818,0.785714,34,0.588235,2,0.5,2,0.5,10,0.6,0,0,0,0,0.625289,0.735308,0.517375,0.358187,0.376,1.553191,6.276596,2024-11-15
"""Zhizhen Zhang""",28,45,1155,0.469388,0.47619,0.529412,18,0.444444,16,0.5625,8,0.5,7,0.285714,0,0,0,0,0.63917,0.736752,0.515106,0.341848,0.384615,1.897959,7.734694,2025-01-01


In [39]:
df_recent.drop("date").write_csv(output_path)